In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, jaccard_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.impute import SimpleImputer
import torch.nn.functional as F
import time

torch.cuda.empty_cache()

df = pd.read_csv('cont_rlm_filled_capped.csv')
df = df.drop('name', axis=1)


X = df.drop('label', axis=1).values
y = df['label'].values


label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

inputsize = X_train.shape[1]

class ANNmodel(nn.Module):
    def _init_(self):
        super(ANNmodel, self)._init_()
        self.ln1 = nn.Linear(inputsize, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.ln2 = nn.Linear(256, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.ln3 = nn.Linear(512, 1024)
        self.bn3 = nn.BatchNorm1d(1024)
        self.ln4 = nn.Linear(1024, 512)
        self.bn4 = nn.BatchNorm1d(512)
        self.ln5 = nn.Linear(512, 256)
        self.bn5 = nn.BatchNorm1d(256)
        self.ln6 = nn.Linear(256, 128)
        self.bn6 = nn.BatchNorm1d(128)
        self.ln7 = nn.Linear(128, 64)
        self.bn7 = nn.BatchNorm1d(64)
        self.ln8 = nn.Linear(64, 4)
        self.dropout = nn.Dropout(0.6)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.ln1(x)), negative_slope=0.01)
        x = F.leaky_relu(self.bn2(self.ln2(x)), negative_slope=0.01)
        x = self.dropout(x)
        x = F.leaky_relu(self.bn3(self.ln3(x)), negative_slope=0.01)
        x = F.leaky_relu(self.bn4(self.ln4(x)), negative_slope=0.01)
        x = self.dropout(x)
        x = F.leaky_relu(self.bn5(self.ln5(x)), negative_slope=0.01)
        x = F.leaky_relu(self.bn6(self.ln6(x)), negative_slope=0.01)
        x = self.dropout(x)
        x = F.leaky_relu(self.bn7(self.ln7(x)), negative_slope=0.01)
        x = torch.sigmoid(self.ln8(x))
        return x



def save_best_model(model, path):
    torch.save(model.state_dict(), path)

def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=100, patience=10):
    best_loss = float('inf')
    patience_counter = 0
    model.train()
    start_time = time.time()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(dataloaders['train'].dataset)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in dataloaders['val']:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
        val_loss /= len(dataloaders['val'].dataset)
        print(f'Validation Loss: {val_loss:.4f}')
        scheduler.step(val_loss)
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            save_best_model(model, 'ANN_best_model_ManuelModel.pt')
        else:
            patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping")
            break
    end_time = time.time()
    print(f'Training Time: {end_time - start_time:.2f} seconds')
    model.load_state_dict(torch.load('ANN_best_model_ManuelModel.pt'))
    return model

def evaluate_model(model, dataloaders):
    model.eval()
    y_true = []
    y_pred = []
    y_scores = []
    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_scores.extend(F.softmax(outputs, dim=1).cpu().numpy())
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_scores = np.array(y_scores)
    print("Classification Report:")
    print(classification_report(y_true, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print("Accuracy Score:", accuracy_score(y_true, y_pred))
    print("Precision Score:", precision_score(y_true, y_pred, average='weighted'))
    print("Recall Score:", recall_score(y_true, y_pred, average='weighted'))
    print("F1 Score:", f1_score(y_true, y_pred, average='weighted'))
    print("Jaccard Score:", jaccard_score(y_true, y_pred, average='weighted'))


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ANNmodel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)

dataloaders = {
    'train': train_loader,
    'val': test_loader
}

model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=500, patience=50)
evaluate_model(model, dataloaders)

ValueError: optimizer got an empty parameter list

In [ ]:
import matplotlib.pyplot as plt

# Model tanımının küçük bir düzeltme ile doğru çalışması için aşağıdaki şekilde olmalıdır:
class ANNmodel(nn.Module):
    def __init__(self):
        super(ANNmodel, self).__init__()
        self.ln1 = nn.Linear(inputsize, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.ln2 = nn.Linear(256, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.ln3 = nn.Linear(512, 1024)
        self.bn3 = nn.BatchNorm1d(1024)
        self.ln4 = nn.Linear(1024, 512)
        self.bn4 = nn.BatchNorm1d(512)
        self.ln5 = nn.Linear(512, 256)
        self.bn5 = nn.BatchNorm1d(256)
        self.ln6 = nn.Linear(256, 128)
        self.bn6 = nn.BatchNorm1d(128)
        self.ln7 = nn.Linear(128, 64)
        self.bn7 = nn.BatchNorm1d(64)
        self.ln8 = nn.Linear(64, 4)
        self.dropout = nn.Dropout(0.6)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.ln1(x)), negative_slope=0.01)
        x = F.leaky_relu(self.bn2(self.ln2(x)), negative_slope=0.01)
        x = self.dropout(x)
        x = F.leaky_relu(self.bn3(self.ln3(x)), negative_slope=0.01)
        x = F.leaky_relu(self.bn4(self.ln4(x)), negative_slope=0.01)
        x = self.dropout(x)
        x = F.leaky_relu(self.bn5(self.ln5(x)), negative_slope=0.01)
        x = F.leaky_relu(self.bn6(self.ln6(x)), negative_slope=0.01)
        x = self.dropout(x)
        x = F.leaky_relu(self.bn7(self.ln7(x)), negative_slope=0.01)
        x = torch.sigmoid(self.ln8(x))
        return x

# Özellik önemini hesaplayan ve görselleştiren fonksiyon
def plot_feature_importance(model, feature_names):
    # Modelin ilk katmanındaki ağırlıkları al
    weights = model.ln1.weight.cpu().detach().numpy()
    # Ağırlıkların ortalamasını alarak önem derecelerini hesapla
    importance = np.mean(np.abs(weights), axis=0)
    
    # Özellik önemlerini görselleştir
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(importance)), importance, tick_label=feature_names)
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.title('Feature Importance')
    plt.xticks(rotation=90)
    plt.show()

# Modeli eğitildikten sonra çağır
feature_names = df.drop('label', axis=1).columns
plot_feature_importance(model, feature_names)
